In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%run figures.py 'hta_2deg.nc' -c volume

In [2]:
import matplotlib as mpl
from matplotlib import pyplot as plt
from cmcrameri import cm
import numpy as np
import copy
import string
mpl.rcParams.update(mpl.rcParamsDefault)

mpl.rcParams['axes.linewidth'] = 3
mpl.style.use('seaborn-poster')

path_15 = home / 'work' / 'data' / 'hydrothermal_alteration' / 'hta_15deg.nc'
data_15 = {}
data_15.update({'volume': xr.open_dataset(path_15, group='volume')})

time_steps = 25
i = 0
file_num=0
fontsize = 24
time = 21


fig, axes = plt.subplots(1,2, figsize=(13,9), gridspec_kw={'width_ratios': [3, 1]})
axis = axes[0]

category = 'volume'
plot_vars = ['Anhydrite'] * 4

range_sets = [(19,23)]
labels = [[7, 8, 9, 10, 11]]

linestyles = ['-', '--']
x_labels = ['[Mg$\mathbf{^{2+}}$] / mM', '[Ca$\mathbf{^{2+}}$] / mM', '[SO$\mathbf{_4}}$] / mM', '[HCO$\mathbf{_3}^-$] / mM']

cols = ['r', 'b', 'g', 'm', 'k', 'c']
temperatures = [2, 15]

legend_titles=[1,2,3,4]

j=0

#Cell vol /cm^3
cell_vol = 200 * 100**2
#Calcite molar volume cm^3/mol
anhyd_mv = 46.010
vf_to_mols = cell_vol / anhyd_mv

scale_factors = (vf_to_mols, vf_to_mols, vf_to_mols, vf_to_mols)

axis.grid(False)
axis.tick_params(length=8, width=4)
axis.set_ylabel('Global S flux into crust / Tmol yr$\mathbf{^{-1}}$', fontweight='bold')
axis.set_xlabel('pH', fontweight='bold')
axis.text(-0.05, 1.05, f'({string.ascii_uppercase[0]})', transform=axis.transAxes, size=20, weight='bold')
for tick in axis.xaxis.get_major_ticks():
    tick.label1.set_fontweight('bold')
    tick.label2.set_fontweight('bold')
for tick in axis.yaxis.get_major_ticks():
    tick.label1.set_fontweight('bold')
    tick.label2.set_fontweight('bold')
#axis.invert_yaxis()

leg_entries = ['2', '15']

datasets = [data, data_15]

for dataset, ls, temperature in zip(datasets, linestyles, temperatures):

    iterables = zip(np.arange(len(plot_vars)), plot_vars, legend_titles, scale_factors, range_sets, labels, leg_entries)
    # Axis level loop
    for j, plot_var, legend_title, scale, range_set, label_a, leg_entry in iterables:
        # Line set loop (i.e. groups of lines sharing a property)
        # Line instance loop
        index = np.arange(range_set[0], range_set[1]+1)
        y = []
        x = copy.deepcopy(label_a)
        pop_list = []

        for n, ind in enumerate(index):
            try:
                val = dataset[category].isel(file_num = ind, time=time, Y=0, Z=0)[plot_var].sum(dim='X').to_numpy() * scale
                if val == 0:
                    pop_list.append(n)
                else:
                    y.append(val)
            except:
                pop_list.append(n)
                continue


        pop_list.sort(reverse=True)
        print(pop_list)
        for q in pop_list:
            x.pop(q)

        axis.plot(x, s_moles_to_flux(y), label= f'{temperature}', linestyle=ls, c='k')

        # Axis decorations
        axis.legend(title='Temperature / °C', prop=font_props, title_fontproperties=font_props, framealpha=0, loc=2)
        axis.set_ylim(0,1)




In [3]:
scale_factors = (1,1,1,1)

axis = axes[1]

#range_sets = [(1,6), (7,10), (11,15), (16,20)]
# Temporarily change range set to account for missing file at start.
range_sets = [(19,23)]
labels = [7, 8, 9, 10, 11],

linestyles = ['-', '--']
legend_titles = ['pH']

x_lims= [(0,10), (0,6), (0,6), (0,6)]
cols = ['r', 'b', 'g', 'm', 'k', 'c']
temperatures = [2, 15]

iterables = zip(plot_vars, axes, legend_titles, scale_factors, range_sets, labels)

j=0

#Prep fig before plotting
axis.xaxis.tick_top()
axis.xaxis.set_label_position('top')
axis.grid(False)
axis.tick_params(length=8, width=4)
axis.set_xlabel(f'{plot_vars[0]} / vol. frac.', fontweight='bold')
axis.set_ylabel('Depth / m', fontweight='bold')
#axis.set_ylim(0,200)
#axis.set_xlim(0,45)
for tick in axis.xaxis.get_major_ticks():
    tick.label2.set_fontweight('bold')
for tick in axis.yaxis.get_major_ticks():
    tick.label1.set_fontweight('bold')
axis.invert_yaxis()

datasets = [data]

for dataset, ls, temperature in zip(datasets, linestyles, temperatures):

    iterables = zip(range(len(plot_vars)), plot_vars, legend_titles, scale_factors, range_sets, labels)
    # Axis level loop
    for j, plot_var, legend_title, scale, range_set, label_a in iterables:
        axis.text(-0.275, 1.05, '({})'.format(string.ascii_uppercase[1]), transform=axis.transAxes, size=20, weight='bold')
        # Line set loop (i.e. groups of lines sharing a property)
        # Line instance loop
        index = np.arange(range_set[0]-1, range_set[1])
        for ind, label, c in zip(index, label_a, cols):
            if i == ind:
                axis.set_prop_cycle(None)
            try:
                x = dataset['volume'].isel(file_num = ind, time=time, Y=0, Z=0)[plot_var].values * scale
                y = dataset['volume'].isel(file_num = ind, time=time, Y=0, Z=0)['X'].values
            except IndexError:
                x=0
                y=0
            axis.plot(x, y, label=f'{label}', linestyle=ls, c=c)
            leg = axis.legend(title=legend_title, prop={'weight': 'bold', 'size': 15}, title_fontproperties={'weight': 'bold', 'size': 15}, framealpha=0, loc='upper right')
            leg.get_title().set_multialignment('center')

fig.tight_layout()

fig.savefig('output/anhydrite_v_ph.png', dpi=300)
fig.show()